# Hand load evaluation on truss element

In [43]:
%reset -f
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym




Define node points with x,y coordinate and the distance between points

In [56]:
#Define all the nodes
nodes = {   
    "P1" : np.array([0,0]),
    "P2" : np.array([-24,31]),
    "P3" : np.array([18.69,119.5]),
    "P4" : np.array([-24,135.25]),
    "P5" : np.array([20,165]),
    "P6" : np.array([-24,185.23]),
    "P7" : np.array([-24,225.53]),
    "P8" : np.array([40.40,-2.88])
 }

#Calculate distance between nodes and unit vector
distances = {}
versors ={}

for name1, coordinate1 in nodes.items():
    for name2, coordinate2 in nodes.items():
        if name1 != name2:
            distance_name = f"{name1}{name2}"  # Naming distances
            versor_name = f"n{name1}{name2}"   # Naming versors

            dist = np.linalg.norm(coordinate1 - coordinate2)
            distances[distance_name] = dist

            versors[versor_name] = (coordinate1-coordinate2)/dist
            
#To acess to the variable use this notation
print(distances["P1P2"])
print(versors["nP1P2"])

39.20459156782532
[ 0.61217319 -0.79072371]


: 

: 